In [1]:
import torch
import torch.nn as nn
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import torch.optim as optim
from ANN import *

In [2]:
df_cleaned=pd.read_csv('Processed_DATA_PITSTRAT.csv')

def preprocess(df_cleaned):

    input_data = df_cleaned[['circuitId', 'raceId', 'points', 'lap','position','laps']].values
    target_data = df_cleaned['stop'].values

    scaler = StandardScaler()
    input_data[:, 2:] = scaler.fit_transform(input_data[:, 2:])

    circuit_encoded = F.one_hot(torch.tensor(input_data[:, 0],dtype=torch.long), num_classes=81)
    race_encoded = F.one_hot(torch.tensor(input_data[:, 1],dtype=torch.long), num_classes=1133)
    points = torch.tensor(input_data[:, 2], dtype=torch.float32).unsqueeze(1)
    lap = torch.tensor(input_data[:, 3], dtype=torch.float32).unsqueeze(1)
    position = torch.tensor(input_data[:, 4], dtype=torch.float32).unsqueeze(1)
    laps = torch.tensor(input_data[:, 5], dtype=torch.float32).unsqueeze(1)


    input_data_encoded = torch.cat([circuit_encoded, race_encoded, points, lap,position,laps], dim=1)
    target_encoded = F.one_hot(torch.tensor(target_data,dtype=torch.long), num_classes=8)
    return input_data_encoded,target_encoded


input_data_encoded,target_encoded=preprocess(df_cleaned)
X_train, X_test, y_train, y_test = train_test_split(input_data_encoded, target_encoded, test_size=0.2, random_state=42)
train_dataset = F1Dataset(X_train, y_train)
test_dataset = F1Dataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
best_loss = float('inf')
circuit_id_max = 81
race_id_max = 1133
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
weight_decay = 1e-5
criterion = nn.CrossEntropyLoss()



cpu


In [4]:
Pitstrat_predictor_model= MLPModel(circuit_id_max=circuit_id_max, race_id_max=race_id_max,dropout_prob=0.2).to(device)
state_dict=(torch.load('../Pitstrat/best_model.pth',map_location=torch.device('cpu')))
Pitstrat_predictor_model.load_state_dict(state_dict)

C:\Users\user\AppData\Local\Temp\ipykernel_14996\1289532668.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict=(torch.load('../Pitstrat/best_model.pth',map_locat

<All keys matched successfully>

In [8]:
import joblib
lr = joblib.load('../lr_model.pkl')

In [9]:
def align_features(X_train, new_input):
    """
    Align new input features with the training feature space.
    Missing columns will be filled with 0.
    """
    # Identify missing columns between the training and new input
    missing_cols = list(set(X_train.columns) - set(new_input.columns))
    
    # Add missing columns (initialized with 0)
    missing_df = pd.DataFrame(0, index=new_input.index, columns=missing_cols)

    # Combine the original input with the missing columns
    aligned_input = pd.concat([new_input, missing_df], axis=1)

    # Ensure the order of columns matches the training data
    aligned_input = aligned_input[X_train.columns]

    return aligned_input

In [18]:
df_lr=pd.read_csv('../data_v1/df_training.csv')

In [19]:
from sklearn.model_selection import train_test_split
X = df_lr.drop(['positionOrder'], axis=1)
Y = df_lr['positionOrder']

In [20]:
X_train_lr, X_test_lr, Y_train_lr, Y_test_lr = train_test_split(X, Y, test_size=0.2, random_state=42)

In [21]:
new_race = {
    'grid': [4, 2,1,6,5,7,11,8,13,10,12,18,3,16,15,21,22,17,20,14,9,19],
    'laps': [57] * 22,  # Same laps for all drivers
    'circuitId_3': [1] * 22  # Circuit ID 3 for all drivers
}

# One-hot encode driver and constructor IDs
drivers = [8,13,9,2,5,15,17,3,10,4,22,21,1,6,7,19,11,14,16,12,18,20]
constructors = [6,6,2,2,1,7,9,3,7,4,11,10,1,3,5,8,8,9,10,4,11,5]

# Add one-hot encoded driver and constructor columns
for driver in drivers:
    new_race[f'driverId_{driver}'] = [1 if i == drivers.index(driver) else 0 for i in range(22)]

for constructor in constructors:
    new_race[f'constructorId_{constructor}'] = [1 if i == constructors.index(constructor) else 0 for i in range(22)]


new_race_df = pd.DataFrame(new_race)


aligned_input = align_features(X_train_lr, new_race_df)


predictions = lr.predict(aligned_input)

print(predictions)
winner_index = predictions.argmax()  
winning_driver = drivers[winner_index]

print(f"The predicted winner is Driver ID: {winning_driver}")

[0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
The predicted winner is Driver ID: 1
